<a href="https://colab.research.google.com/github/maahirarubaiya/Ramonify-Personal-Finance-Dashboard/blob/main/Ramonify_Gradio_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip -q install gradio pandas numpy matplotlib scikit-learn

In [5]:
from google.colab import files

uploaded = files.upload()


Saving transactions.csv to transactions.csv


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import gradio as gr
import tempfile
from pathlib import Path

def analyze_transactions(file, overspend_threshold_pct, cut_pct, monthly_goal):
    # ---------- Load & clean ----------
    df = pd.read_csv(file.name)

    required = {"date", "amount", "category", "merchant", "type"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {sorted(missing)}")

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["amount"] = pd.to_numeric(df["amount"], errors="coerce")
    df["type"] = df["type"].astype(str).str.lower().str.strip()

    if df["date"].isna().any():
        raise ValueError("Some dates could not be parsed. Use YYYY-MM-DD format.")
    if df["amount"].isna().any():
        raise ValueError("Some amounts are not numeric. Fix the amount column.")
    if not set(df["type"].unique()).issubset({"income", "expense"}):
        raise ValueError('type must contain only "income" or "expense".')

    df["month"] = df["date"].dt.to_period("M").astype(str)

    # ---------- Monthly summary ----------
    monthly_summary = (
        df.groupby(["month", "type"])["amount"]
          .sum()
          .unstack(fill_value=0)
          .reset_index()
          .sort_values("month")
          .reset_index(drop=True)
    )

    if "income" not in monthly_summary.columns:
        monthly_summary["income"] = 0.0
    if "expense" not in monthly_summary.columns:
        monthly_summary["expense"] = 0.0

    monthly_summary["net"] = monthly_summary["income"] - monthly_summary["expense"]
    monthly_summary[["income", "expense", "net"]] = monthly_summary[["income", "expense", "net"]].round(2)

    # ---------- Category analysis ----------
    expenses = df[df["type"] == "expense"].copy()

    monthly_category = (
        expenses.groupby(["month", "category"])["amount"]
                .sum()
                .reset_index()
                .sort_values(["month", "amount"], ascending=[True, False])
    )
    monthly_category["amount"] = monthly_category["amount"].round(2)

    top3_by_month = (
        monthly_category.groupby("month")
                        .head(3)
                        .reset_index(drop=True)
    )

    # ---------- Forecasting ----------
    monthly_summary["rolling_avg_3"] = monthly_summary["net"].rolling(window=3).mean().round(2)
    rolling_forecast = monthly_summary["rolling_avg_3"].iloc[-1]

    X = np.arange(len(monthly_summary)).reshape(-1, 1)
    y = monthly_summary["net"].values
    model = LinearRegression()
    model.fit(X, y)
    regression_forecast = float(model.predict(np.array([[len(monthly_summary)]]))[0])

    forecast_df = pd.DataFrame({
        "method": ["rolling_average_3_month", "linear_regression_trend"],
        "predicted_next_month_net": [rolling_forecast, regression_forecast]
    })
    forecast_df["predicted_next_month_net"] = forecast_df["predicted_next_month_net"].round(2)

    # ---------- Overspending warning (color banner) ----------
    last_month = monthly_summary["month"].iloc[-1]
    last_expense = float(monthly_summary["expense"].iloc[-1])
    last_income = float(monthly_summary["income"].iloc[-1])
    last_net = float(monthly_summary["net"].iloc[-1])

    # Compare last month spending to prior average (if enough history)
    prev_avg_expense = None
    ratio = 1.0
    if len(monthly_summary) >= 4:
        prev_avg_expense = float(monthly_summary["expense"].iloc[:-1].mean())
        ratio = last_expense / prev_avg_expense if prev_avg_expense > 0 else 1.0

    # Threshold slider: e.g., 25% => ratio >= 1.25 is red warning
    threshold_ratio = 1.0 + (overspend_threshold_pct / 100.0)

    if last_net < 0 or (prev_avg_expense is not None and ratio >= threshold_ratio):
        banner_color = "#7f1d1d"   # red
        banner_text = "⚠️ Spending Alert"
        sub_text = "Your spending looks high. You may run low on money next month."
    elif prev_avg_expense is not None and ratio >= 1.0 + max(0.10, overspend_threshold_pct/200.0):
        banner_color = "#78350f"   # amber
        banner_text = "⚠️ Spending Watch"
        sub_text = "Your spending is a bit higher than usual. Small cuts can help."
    else:
        banner_color = "#14532d"   # green
        banner_text = "✅ Spending Looks OK"
        sub_text = "Your spending looks close to your usual pattern."

    if prev_avg_expense is not None:
        sub_text += f" (This month: ${last_expense:,.2f} | Prior avg: ${prev_avg_expense:,.2f} | Threshold: {overspend_threshold_pct:.0f}% )"

    warning_html = f"""
    <div style="padding:12px;border-radius:12px;color:white;background:{banner_color};">
      <div style="font-size:16px;font-weight:700;margin-bottom:4px;">{banner_text}</div>
      <div style="font-size:13px;opacity:0.95;">{sub_text}</div>
    </div>
    """

    # ---------- Tips + Goal Plan ----------
    last_month_cats = monthly_category[monthly_category["month"] == last_month]

    if len(last_month_cats) > 0:
        top_cat = str(last_month_cats.iloc[0]["category"])
        top_cat_amt = float(last_month_cats.iloc[0]["amount"])
    else:
        top_cat = "N/A"
        top_cat_amt = 0.0

    # Convert cut% dropdown into fraction
    cut_fraction = float(cut_pct) / 100.0
    savings_from_cut = top_cat_amt * cut_fraction

    # Goal planning: how much more net needed to hit goal?
    # If monthly_goal is blank/0, keep it simple.
    goal_lines = []
    try:
        goal = float(monthly_goal) if monthly_goal not in [None, ""] else 0.0
    except:
        goal = 0.0

    if goal > 0:
        if last_net >= goal:
            goal_lines.append(f"Goal check: You already met your goal of ${goal:,.2f} this month. Nice!")
        else:
            gap = goal - last_net
            goal_lines.append(f"Goal check: To reach ${goal:,.2f} left over, you need about ${gap:,.2f} more next month.")
            if top_cat != "N/A":
                # How much percent cut in top category would cover the gap?
                required_cut_pct = (gap / top_cat_amt) * 100 if top_cat_amt > 0 else None
                if required_cut_pct is not None:
                    required_cut_pct = min(required_cut_pct, 100.0)
                    goal_lines.append(f"One option: reduce {top_cat} by about {required_cut_pct:.0f}% to cover the gap (if possible).")
            goal_lines.append("You can also increase income (extra shift, tutoring) or cut 2 categories slightly.")

    rolling_pred = float(forecast_df.loc[forecast_df["method"]=="rolling_average_3_month", "predicted_next_month_net"].iloc[0])

    insights_lines = [
        f"Month analyzed: {last_month}",
        f"You earned ${last_income:,.2f} and spent ${last_expense:,.2f}.",
        f"Money left after spending: ${last_net:,.2f}.",
    ]

    if top_cat != "N/A":
        insights_lines.append(f"Biggest spending area: {top_cat} (${top_cat_amt:,.2f}).")
        insights_lines.append(f"Tip: If you cut {top_cat} by {int(cut_pct)}%, you could save about ${savings_from_cut:,.2f} next month.")
    else:
        insights_lines.append("Tip: Add more categorized expenses to get personalized tips.")

    insights_lines.append(f"Next month estimate: around ${rolling_pred:,.2f} left if your habits stay similar.")

    # Add goal plan lines (if any)
    insights_lines.extend(goal_lines)

    insights = "\n".join([f"• {x}" for x in insights_lines])

    # ---------- Plot ----------
    monthly_summary["month_dt"] = pd.to_datetime(monthly_summary["month"])
    fig = plt.figure()
    plt.plot(monthly_summary["month_dt"], monthly_summary["net"], marker="o")
    plt.axhline(0)
    plt.title("Monthly Net Cash Flow")
    plt.xlabel("Month")
    plt.ylabel("Net (Income - Expense)")
    plt.xticks(rotation=45)
    plt.tight_layout()

    # ---------- Create downloadable CSV files ----------
    tmpdir = Path(tempfile.mkdtemp())

    summary_path = tmpdir / "monthly_summary.csv"
    top3_path = tmpdir / "top3_by_month.csv"
    forecast_path = tmpdir / "forecast_results.csv"

    monthly_summary.drop(columns=["month_dt"], errors="ignore").to_csv(summary_path, index=False)
    top3_by_month.to_csv(top3_path, index=False)
    forecast_df.to_csv(forecast_path, index=False)

    return (
        fig,
        monthly_summary.drop(columns=["month_dt"], errors="ignore"),
        top3_by_month,
        forecast_df,
        insights,
        warning_html,
        str(summary_path),
        str(top3_path),
        str(forecast_path),
    )


demo = gr.Interface(
    fn=analyze_transactions,
    inputs=[
        gr.File(label="Upload transactions.csv"),
        gr.Slider(minimum=10, maximum=50, value=25, step=5, label="Overspending warning threshold (%)"),
        gr.Dropdown(choices=[5, 10, 15, 20, 25], value=10, label="Tip: cut biggest category by (%)"),
        gr.Number(value=0, label="Monthly savings goal ($)"),
    ],
    outputs=[
        gr.Plot(label="Monthly Net Cash Flow"),
        gr.Dataframe(label="Monthly Summary"),
        gr.Dataframe(label="Top 3 Spending Categories per Month"),
        gr.Dataframe(label="Forecast Results"),
        gr.Textbox(label="Simple Insights & Tips", lines=10),
        gr.HTML(label="Spending Warning"),
        gr.File(label="Download: monthly_summary.csv"),
        gr.File(label="Download: top3_by_month.csv"),
        gr.File(label="Download: forecast_results.csv"),
    ],
    title="Ramonify — Personal Finance Dashboard",
    description="Upload a transaction CSV to see cash flow, top spending categories, forecasts, tips, warnings, and download results."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://931cf0aaa07b4d18b9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
